In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload

import numpy as np
import cv2
import math
import csv
import pandas as pd

In [5]:
# trains = pd.read_csv('Trains.csv', index_col=0)

In [12]:
# trains.shape
# trains.head()

,0,1,2,3,4,5,6,7,8,9,...,16373,16374,16375,16376,16377,16378,16379,16380,16381,16382
0,3.151267,3.185333,106.462600,153.180633,728.949000,730.185800,754.049000,1194.196333,1229.704933,1486.291133,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9.336600,15.777833,25.908133,26.151167,31.067833,35.569133,36.504400,47.373400,50.728367,51.963600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.017333,6.025767,6.041500,6.059200,6.063500,6.090333,9.678833,12.841267,12.854967,12.876667,...,4180.966133,4180.975667,4180.984600,4180.992467,4181.055633,4181.059733,4181.068033,4181.097933,4181.105800,4181.244667
3,3.828100,5.340033,7.091367,7.245833,8.541600,8.888833,9.468867,9.942133,10.359133,11.888200,...,2764.472933,2764.758767,2765.086133,2765.249400,2765.846833,2765.861900,2765.884933,2766.062400,2766.215367,2766.314333
4,432.656967,669.366733,818.183167,1122.724267,1161.551433,1182.497033,1227.039000,1365.555667,1458.530000,1585.273833,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
# begin_time = min(trains.min())
# end_time = max(trains.max())+1
# print(begin_time, end_time)

0.0237 8290.087033


In [2]:
def HSE_detection(Spikes, ori_bins=30000, bins=100):
    '''
    spikes: list object [cell, timebins]
    '''
    # clustered firing rate
    cell_num = Spikes.shape[0]
    ratio = int(ori_bins / bins)
    time_Bins = math.ceil(Spikes.shape[1] / ratio)
    spikes = np.zeros((Spikes.shape[0], ratio*time_Bins))
    spikes[0:Spikes.shape[0], 0:Spikes.shape[1]] = Spikes
    spikes = np.mean(spikes.reshape(spikes.shape[0], ratio, time_Bins), axis=1)
    cluster_fr = np.mean(spikes, axis=0)
    cluster_sp = np.round(cluster_fr/bins, 0)
    
    syn_spike_rate = np.zeros(time_Bins)
    gaussian_kernel = np.array(cv2.getGaussianKernel(15, 15)).reshape(15)
    a = cluster_fr[0]
    b = cluster_fr[-1]
    cell_activity = np.array([a for i in range(7)] + list(cluster_fr) + [b for i in range(7)])
    for i in range(7, time_Bins-1):
        syn_spike_rate[i] = np.sum(gaussian_kernel * cell_activity[i-7: i+8])
    
    mean = np.mean(syn_spike_rate)
    SD = np.std(syn_spike_rate)
    events_time = []
    # find HSE
    t = 0
    t_begin = None
    t_end = None
    while t < time_Bins:
        while t < time_Bins and syn_spike_rate[t] <= mean + 3*SD:
            t += 1
        if t < time_Bins:
            t_begin = t
        else:
            break
        while t < time_Bins and syn_spike_rate[t] >= mean:
            t += 1
        if t < time_Bins:
            t_end = t
        else:
            t_end = time_Bins - 1
        if 75 <= t_end - t_begin <= 750:
            events_time.append([t_begin, t_end])
        t = t_end + 1

    # reject HES: fewer than 5 spikes, 4 cells or with less than 10% of the population of neurons active
    i = 0
    while i < len(events_time):
        begin, end = events_time[i]
        # fewer than 5 spikes
        if np.sum(cluster_sp[begin: end]) < 5:
            del events_time[i]
        # fewer than 4 cells or less than 10% of the population of neurons active
        num = 0
        for i in range(cell_num):
            if np.max(spikes[i][begin: end]) > 0:
                num += 1
        if num < 4 or num < cell_num * 0.1:
            del events_time[i]
        i += 1
    
    # store spike trains for HSE
    spike_train = [[[] for i in range(Spikes.shape[0])] for j in range(len(events_time))]
    for j in range(len(events_time)):
        t_begin, t_end = events_time[j]
        for i in range(Spikes.shape[0]):
            spike_train[j][i].append(Spikes[i][t_begin*ratio: t_end * ratio])
    
    return mean, SD, syn_spike_rate, np.array(events_time)

In [44]:
# HSE_detection(trains)

array([[3.0700000e-02, 1.4270000e-01],
       [1.8670000e-01, 2.8170000e-01],
       [1.8670000e-01, 3.2370000e-01],
       [1.8670000e-01, 3.5870000e-01],
       [1.8670000e-01, 3.8670000e-01],
       [1.8670000e-01, 4.7170000e-01],
       [1.8670000e-01, 5.2870000e-01],
       [1.8670000e-01, 5.6270000e-01],
       [1.8670000e-01, 5.9670000e-01],
       [1.8670000e-01, 6.2070000e-01],
       [1.8670000e-01, 6.8770000e-01],
       [1.8670000e-01, 7.0470000e-01],
       [1.8670000e-01, 7.8270000e-01],
       [8.1870000e-01, 9.4070000e-01],
       [8.1870000e-01, 9.6970000e-01],
       [1.0527000e+00, 1.2017000e+00],
       [1.0527000e+00, 1.2247000e+00],
       [1.0527000e+00, 1.2457000e+00],
       [1.0527000e+00, 1.2617000e+00],
       [1.0527000e+00, 1.3117000e+00],
       [1.0527000e+00, 1.3637000e+00],
       [1.0527000e+00, 1.4117000e+00],
       [1.0527000e+00, 1.4417000e+00],
       [1.0527000e+00, 1.5087000e+00],
       [1.0527000e+00, 1.5467000e+00],
       [1.0527000e+00, 1.